In [2]:
# Pick convention: 
# We work on a rectangular cell of side lengths a, b, c
# This cuboid is in the positive coordinates and centered at 0
# I.e. spanned by (a,0,0), (0,b,0) and (0,0,c)

In [4]:
import numpy as np
import random as rd

# Generating random points (optional)

This function generates $N$ random points in a cell of sidelenghts $a,b,c$.  
This is for demonstration and testing, but not an integral part of the software.

In [5]:
def gen_points(N=5,a=1,b=1,c=1):
    """
    Generates N random points in [0,a]x[0,b]x[0,c] cuboid,
    outputts Nx3 numpy array, each row containing x, y,z coordinates of a point.
    """
    return np.array([[rd.random(),rd.random(),rd.random()] for i in range(N)])


In [6]:
gen_points().shape

(5, 3)

# Creating periodic $\alpha$-filtration

## Auxiliary functions for periodic_filtration()

### Duplicating points to neighbouring cells
Given points in a unit cell, this function duplicates them by translating to all neighbouring cells.  
This is an auxiliary step to calculating the _periodic_ $\alpha$-filtration. 

In [ ]:
def torus_copy(points, a=1,b=1,c=1):
    """
    Takes numpy array points of N points on axbxc cube
    and creates 8 copies surrounding it.
    
    Returns new_points, containing original points and 9+8+9=26 offset copies,
    so Nx27 points in total.
    """
    
    N = np.shape(points)[0]
    # first index  = which axbxc cuboid we are working on
    # second index = index of corresponding original point
    # third index  = x, y, z coordinates of point
    new_points = np.zeros((27,N,3)) 

    i = 0 # to index the cuboid
    for x in [-a,0,a]:
        for y in [-b,0,b]:
            for z in [-c,0,c]:
            transl = np.array([[x,y,z] for dummy in range(N)])
            new_points[i,:,:] = points + transl
            i += 1
            
    return new_points.reshape(-1,3)

### Other functions

In [ ]:

def dim_split(filt):
    """
    Takes list of simplices 'filt' with filtration values, 
    i.e. with elements ([3,2], 0.123)
    Splits filt by dimension of elements.
    Outputs the 4 seperate lists, one for each dimension 0 to 3.
    """
    simp_0 = []
    simp_1 = []
    simp_2 = []
    simp_3 = []
    
    for (simp, val) in filt:
        p = len(simp)-1
        
        if p == 0:
            simp_0.append((simp,val))
        if p == 1:
            simp_1.append((simp,val))
        if p == 2:
            simp_2.append((simp,val))
        if p == 3:
            simp_3.append((simp,val))
    return simp_0, simp_1, simp_2, simp_3

In [ ]:
def check_bound(point, a=1, b=1, c=1):
    """
    Takes point with x,y,z coordinates.
    Checks if point lies in cuboid [0,a) x [0,b) x [0,c).
    If yes, outputs True, else False.
    """
    x = point[0]
    y = point[1]
    z = point[2]
    
    return (    x >= 0 and x < a 
            and y >= 0 and y < b 
            and z >= 0 and z < 1)

In [ ]:
def equiv_num(x1,x2,m=1):
    """
    Calculates 'x1-x2 mod m', where we always want to take 
    the representative with the smallest absolute value. 
    """
    z1 = abs((x1 - x2)%m)
    z2 = abs(z1 - m)
    return min(z1,z2)

In [ ]:
def identification_list(S0_list, S0, eps = 1e-5):
    """
    Input:
        S0_list ... naive filtration list  of 0-simplices ([3],3) as given by gudhi
        S0      ... list of Simplex0D objects generated from S0

    Output:
        identify_list ... list of lists with entries [i,j], 
                           meaning that the ith vertex has to be matched to the jth Simplex0D object
    """
    # build identification list for later reference when building higher simplices
    identify_list = []

    for i in range(len(S0_list)):
        simp, filt_value = S0_list[i]
        coord = np.array(alpha_complex.get_point(i))

        for Simplex in S0: # looking for the point in the unit cell this corresponds to              
            other_coord = Simplex.coords

            if ((equiv_num(coord[0],other_coord[0],m=a)<eps) and 
                (equiv_num(coord[1],other_coord[1],m=b)<eps) and
                (equiv_num(coord[2],other_coord[2],m=c)<eps)):

                identify_index = Simplex.int_filt

        identify_list.append([i, identify_index])
    return identify_list


In [16]:
import numpy as np
array = [2,-1,4,100,-1]
ind1 = np.argmin(array)
array.reverse()
ind2 = len(array)-1-np.argmin(array)
if ind1 == ind2:
    print("first vertex is is v{ind1}")
else

In [17]:
def order_vertex(indices, coords):
    """
    Takes two vertices and outputs them by lexicographical order.
    
    Input:
        indices ... list [n0, n1], where ni is the integer filtration index of the ith vertex, identifying it uniquely 
        coords  ... list of lists [[x0, y0, z0], [x1, y1, z1]] containing the coordinates of both points.
        
    Output:
        [nl, nr]   ... list of indices in lexicographical order
        [xl,yl,zl] ... coordinates of left vertex
        [xr,yr,zl] ... coordinates of right vertex 
    
    """
    
    [n0, n1] = indices
    [x0, y0, z0] = coords[0]
    [x1, y1, z1] = coords[1]
    
    
    if x0 < x1 or (x0 == x1 and y0 < y1) or (x0 == x1 and y0 == y1 and z0 < z1):
        # 0 is left
        left  = 0
        right = 1

    else:
        # 1 is left
        left  = 1
        right = 0
    
    # left vertex
    nl = indices[left]
    xl = coord[left][0]
    yl = coord[left][1]

    # right vertex
    nr = indices[right]
    xr = coord[right][0]
    yr = coord[right][1]
    return [nl, nr], [xl,yl,zl], [xr,yr,zl]

### Creation of Simplex objects

In [ ]:
def create_S0(S0_list):
    """
    Input:
        S0_list ... List of 0-simplices as generated by gudhi, 
        after having split them by dimension
    Output: 
        S0 ... List of Simplex0D objects containing the combinatorical representation,
        the integer filtration value, the continuous filtration value 
        and the geometric coordinate of the point
    """

    int_filt_value = 0
    S0 = []

    for i in range(len(simp0)):
        simp, filt_value = simp0[i]

        # check if 0-simplex lies in main cell
        coord = alpha_complex.get_point(i)
        if check_bound(coord,a=a,b=b,c=c): # point lies inside main cell
            S0.append(sc.Simplex0D([int_filt_value], 
                                            int_filt_value, 
                                            filt_value, 
                                            coord))
            int_filt_value +=1
    return S0

In [ ]:
def create_S1(S1_list, S0, alpha_complex, identify_list):
    """
    Input: 
        Input:
        S1_list ... List of 1-simplices as generated by gudhi, 
        after having split them by dimension
        S0 ... List of Simplex0D objects of the filtration
        alpha_complex ... alpha_complex objects as given by gudhi
        identify_list ... identification list for periodically copied points as given by identification_list()
        
    Output: 
        S1 ... List of Simplex1D objects containing the combinatorical representation,
        the integer filtration value, the continuous filtration value,
        the crossing vector as numpy array and the vertices constituting the boundary
        in lexicographical order
    """


    # note that this is not the actual integer filtration value 
    # and this will get corrected later on
    int_filt_value = len(S0) 
    S1 = []


    for i in range(len(S1_list)):
        """
        This does not work in 3d, because for a total order we need more than left-right
        !!!!!!!
        """
        simp, filt_value = S1_list[i]

        # check what the leftmost vertex is
        coord0 = alpha_complex.get_point(n0)
        coord1 = alpha_complex.get_point(n1)

        [nl, nr], coordl,  coordr = order_vertex(simp, [coord0, coord1])
        
        # Check if leftmost vertex is in unit cell
        """
        function "check_bound" has been modified
        this STILL needs to be adjusted here
        also modify for 3d-version
        """
        if check_bound(coordl, xmax=a, ymax=b):
            # Calculate crossing vector
            cross_vec = crossing_vector(coordr,xmax=a,ymax=b)

            # identify the old vertices with the newly numerated ones!!!!! 
            # both for simplex, as well as for ordered vertices

            nl_new = (identify_list[nl])[1]
            nr_new = (identify_list[nr])[1]



            lex_ordered_simp = [S0[nl_new], S0[nr_new]]
            int_ordered_simp = sorted([nl_new,nr_new])

            S1.append(sc.Simplex1D(int_ordered_simp, int_filt_value, filt_value, cross_vec,lex_ordered_simp))

            int_filt_value +=1
    return S1


In [ ]:
def create_S2(S2_list, S1, alpha_complex, identify_list):
    """
    Input: 
        Input:
        S2_list ... List of 2-simplices as generated by gudhi, 
        after having split them by dimension
        S1 ... List of Simplex1D objects of the filtration
        alpha_complex ... alpha_complex objects as given by gudhi
        identify_list ... identification list for periodically copied points as given by identification_list()
        
    Output: 
        S2 ... List of Simplex2D objects containing the combinatorical representation,
        the integer filtration value, the continuous filtration value,
        and the vertices constituting the boundary
        in lexicographical order
    """
    
    """
    !!! DESCIRPTION INCOMPLETE !!!
    """
    
    S2 = []
    for i in range(len(S2_list)):
            simp, filt_value = S2_list[i]

            # how can we find the correct 2-simplices?

            # if we have the vertices of the 2-simplex, we can look for 1-simplices which have two of these boundary points
            # since we only keep 2-simplices with their left-most point in the main cell
            # we can calculate the crossing vectors and then uniquly identify 2 out of 3 boundary elements

            # for the last edge, we have to 
            # - take the middle and the rightmost point,
            # - shift them such that the middle point is now in the main cell
            # - calculate the crossing vector of these shifted points
            # this is then the crossing vector of the last edge, and we can again identify
            # the correct 1-simplex using the vertices and the crossing vector


            [n1, n2, n3] = simp
            coord1 = alpha_complex.get_point(n1)
            coord2 = alpha_complex.get_point(n2)
            coord3 = alpha_complex.get_point(n3)

            # ordering vertices
            a0 = n1
            b0 = n2
            c0 = n3
            a0_coord = coord1
            b0_coord = coord2
            c0_coord = coord3

            [a1,b1], a1_coord, b1_coord = order_vertex([a0,b0], [a0_coord, b0_coord])
            [a2,c1], a2_coord, c1_coord = order_vertex([a1,c0], [a1_coord, c0_coord])
            [b2,c2], b2_coord, c2_coord = order_vertex([b1,c1], [b1_coord, c1_coord])
            
            # a2 is the leftmost, b2 is the middle and c2 is the rightmost coordinate, 
            # or a2 < b2 < c2 in lexicographical order


            # only if a2 is in the main cell do we continue
            if check_bound(a2_coord, xmax=a, ymax=b): 

                # the vertices are not yet in the naming convention we have chosen
                # so we rename them using identify_list
                for i in range(len(identify_list)):
                    new_name = (identify_list[i])[1]
                    if i == a2:
                        a2 = new_name
                    if i == b2:
                        b2 = new_name
                    if i == c2:
                        c2 = new_name


                # first boundary element from a2 to b2
                verts_1  = sorted([a2,b2])
                cr_vec_1 = crossing_vector(b2_coord,xmax=a,ymax=b)

                # second boundary element from a2 to c2
                verts_2  = sorted([a2,c2])
                cr_vec_2 = crossing_vector(c2_coord,xmax=a,ymax=b)

                # third boundary element from b2 to c2
                shift    = crossing_vector(b2_coord,xmax=a,ymax=b)
                verts_3  = sorted([b2,c2])
                cr_vec_3 = crossing_vector([c2_coord[0]-shift[0], c2_coord[1]-shift[1]])

                for j in range(len(S1)):
                    Simplex = S1[j]
                    verts = Simplex.verts
                    cv    = Simplex.cv


                    if (verts[0] == verts_1[0]) and (verts[1] == verts_1[1]) and (np.linalg.norm(cv-cr_vec_1)<eps):
                        bound_1 = Simplex
                    if (verts[0] == verts_2[0]) and (verts[1] == verts_2[1]) and (np.linalg.norm(cv-cr_vec_2)<eps):
                        bound_2 = Simplex
                    if (verts[0] == verts_3[0]) and (verts[1] == verts_3[1]) and (np.linalg.norm(cv-cr_vec_3)<eps):
                        bound_3 = Simplex

                S2.append(sc.Simplex2D( sorted([a2,b2,c2]), int_filt_value, filt_value, [Simplices_0[a2],Simplices_0[b2],Simplices_0[c2]], [bound_1,bound_2,bound_3]))
                int_filt_value += 1
    return S2

In [ ]:
def generate_pfilt(S0,S1,S2,S3):
    """
    Takes the lists of simplex objects Si and generates the 
    mixed-dimension periodic filtration, so with all the simplices, but only saved
    through their vertices and their continuous filtration value.
    """

    periodic_filt = []

    periodic_filt_0 = [(Simplex.verts, Simplex.cont_filt) for Simplex in S0]
    periodic_filt += periodic_filt_0

    periodic_filt_1 = [(Simplex.verts, Simplex.cont_filt) for Simplex in S1]
    periodic_filt += periodic_filt_1

    periodic_filt_2 = [(Simplex.verts, Simplex.cont_filt) for Simplex in S2]
    periodic_filt += periodic_filt_2

    periodic_filt_3 = [(Simplex.verts, Simplex.cont_filt) for Simplex in S3]
    periodic_filt += periodic_filt_3


    periodic_filt.sort(key = lambda some_tuple: some_tuple[1])

    return periodic_filt


In [ ]:
def reorder_by_cont(S0,S1,S2,S3,periodic_filt):
    """
    Since the integer filtration value is wrong (first storted by dimension, 
    only then by actuall filtration value), this needs to be reordered.
    This function takes the Simplex, looks where it is in the periodic filtration,
    and reassignes this placement. 
    """
    for Simplex in S0:
        int_filt = periodic_filt.index((Simplex.verts, Simplex.cont_filt))
        Simplex.update_int_filt(int_filt)
    for Simplex in S1:
        int_filt = periodic_filt.index((Simplex.verts, Simplex.cont_filt))
        Simplex.update_int_filt(int_filt)
    for Simplex in S2:
        int_filt = periodic_filt.index((Simplex.verts, Simplex.cont_filt))
        Simplex.update_int_filt(int_filt)
    for Simplex in S3:
        int_filt = periodic_filt.index((Simplex.verts, Simplex.cont_filt))
        Simplex.update_int_filt(int_filt)
    """
    !!!
    I don't think that I even need a return statement here. 
    """
    return S0, S1, S2, S3


## The final periodic_filtration()

In [ ]:

def periodic_filtration(points, max_alpha_square=float("inf"), a=1, b=1, c=1):
    """
    OLD:
    Outputs
        periodic_filt ... list of tuples, each tuple containing 
            1) simplex encoded as list of (periodically reduced) vertex numbers
            2) filtration value as float
        For example, periodic_filt[22] = ([8, 11], 0.0016)
        
        [Simplices_0, Simplices_1, Simplices_2, Simplices_3], where
        Simplices_0 ... list of 0-simplices encoded as Simplex0D objects
        Simplices_1 ... list of 1-simplices encoded as Simplex1D objects
        Simplices_2 ... list of 2-simplices encoded as Simplex2D objects
        Simplices_3 ... list of 3-simplices encoded as Simplex3D objects
    """
    
    """
    Input:
        a, b, c ... side lengths of cell [0,a] x [0,b] x [0,c]
        points  ... numpy array of size (N,3) containing N points in cell
        max_alpha_square   ... maximum alpha (squared) value of alpha-filtration
        
    Output:
        XXXXXX
    """
    N = points.shape[0]
    points_dup = torus_copy(points)
    
    # alpha-complex generation by gudhi
    alpha_complex = gd.AlphaComplex(points_dup)
    simplex_tree = alpha_complex.create_simplex_tree(max_alpha_square=max_alpha_square)
    filtration = simplex_tree.get_filtration()   
    
    # split unidentified simplices by dimension
    simp0, simp1, simp2, simp3 = dim_split(filtration)
    
 
    
    Simplices_3 = []
    
    """
    I removed the connected component list because I want to generate this as a method later,
    not at initialisation. 
    However, this STILL NEEDS TO BE DONE!!!
    """
    
    
    
    # 0-Simplices ---------------------------------
    S0 = create_S0(simp0)
    identify_list = identification_list(S0_list=simp0, S0=S0, eps = 1e-5)

    
    # 1-Simplices ---------------------------------
    S1 = create_S1(S1_list=simp1, S0=S0, alpha_complex=alpha_complex, identify_list=identify_list)

    
    
    # 2-Simplices ---------------------------------
    S2 = create_S2(S2_list=simp2, S1=S1, alpha_complex=alpha_complex, identify_list=identify_list)
    
    # 3-Simplices
    """
    THIS NEEDS TO BE CREATED
    """

    periodic_filt = generate_pfilt(S0=S0,S1=S1,S2=S2,S3=S3)
    
 
    S0, S1, S2, S3 = reorder_by_cont(S0=S0,S1=S1,S2=S2,S3=S3, periodic_filt=periodic_filt)
    """
    If I don't need a return statement, this would also not need an assignment. 
    See the original definition of the function. 
    """
    

    
    return periodic_filt, [S0, S1, S2, S3]
    


# Calculating PPH

After the calculation of the periodic filtration, next comes calculating the periodic persistent homology. The algorithm here should not differ significantly from the standard ones. The only reason we are writing this from scratch is that pre-existing software is not made to handle our data, especialy in case of degenerate simplices. 

## Auxiliary functions for PH()

In [ ]:
def boundary_matrix(S):
    """
    Takes list of subslists of Simplix type objects.
    Outputs a boundary matrix as numpy matrix.
    
    Note that the boundary matrix is sorted by dimension (their index in S[i]), not as given by .int_filt
    """
    S0, S1, S2, S3 = S
    
    # Initialise matrix of dimension s0+s1+s2(boundaries) x s1+s2+s3
    D = np.zeros((len(S0+S1+S2),len(S0+S1+S2+S3)))
    
    # The 0-Simplices all appear at the beginning and are already ordered by their usual index
    """
    ???
    instead of extracting Si, defining di and calculating D per dimension, 
    can't we do all of this in one go using indices?
    """
    d0 = len(S0)
    d1 = len(S1)
    d2 = len(S2)
    
    for j in range(d1):
        for i in S1[j].verts:
            D[i,j+d0] += 1
    
    for j in range(d2):
        for i in range(d1):
            if S1[i] in S2[j].boundary:
                D[i+d0,j+d0+d1] += 1
    
    """
    ???
    The 3rd boundary matrix is missing at this point. Process should be the same, though.
    """
    
    return np.mod(D,2)

In [ ]:
def check_low(column):
    """ 
    Takes a column i.e. numpy 1dim array. 
    Outputs the index of the lowest non-zero element of the given column.
    If the column is all zero, it returns -1.
    """
    low = -1
    m = len(column)
    for i in range(0,m):
        if column[m-1-i]==1:
            low = m-1-i
            break
    return low


In [ ]:
def column_red(D, exhaustive=False): 
    """
    Takes a binary boundary matrix and performs column reduction on it.
    Returns the reduced matrix "R", as well as a list of its pivot elements "pivots". 
    Zero columns are encoded in pivots as -1. 
    """
    
    
    """
    !!!!
    We want to implement exhaustive reduction at some point. 
    """
    # D is numpy array of size mxn
    (m,n) =  D.shape # rows, column
    R = D.copy()
    V = np.identity(n)
    
    pivots = []
    
    for i in range(0,n):
        # look at current pivot 
        low = check_low(R[:,i])
        
        # check if this is already a pivot element
        while (low in pivots) and (low != -1): 
            # while pivot is taken, perform matrix reduction on R
            
            j = pivots.index(low)
            R[:,i]= (R[:,i] + R[:,j]) % 2
            V[:,i]= (V[:,i] + V[:,j]) % 2
            
            # get new pivot
            low = check_low(R[:,i]) 
            
           
            # don't forget to reduce mod 2
        
        pivots.append(low)
        
    
    return pivots, R, V



In [ ]:
def calc_groups(dim_list, pivots):
    dCn = np.zeros(d+1)
    dZn = np.zeros(d+1)
    dBn = np.zeros(d+1)
    dHn = np.zeros(d+1)

    # Calculate the vector space dimensions, in particular Betti numbers
    m = 0
    for i in range(0,d+1):
        # extract dimensions of vector spaces
        dCn[i]   = dim_list[i]
        dZn[i]   = pivots[m:m+dim_list[i]].count(-1)
        dBn[i-1] = dCn[i] - dZn[i]
        m += dim_list[i]

    for i in range(0,d+1):
        dHn[i]   = dZn[i] - dBn[i]

    return dCn, dZn, dBn, dHn


In [ ]:
def calc_pps(D, dim_list, S):

    pp_int = [] # list of persistence pairs

    # calculate persistence pairs
    for j in range(D.shape[1]):# looking at the jth column

        # determine dimension
        for k in range(len(dim_list)): 
                if j < sum(dim_list[0:k+1]) and j >= sum(dim_list[0:k]):
                    dim = k
        """
        Here we are again taking the numbering first by dimension, then by filtration.
        We need to reference the actuall int_filt of the respective simplices. 
        This should work by looking at the jth k-simplex and taking its int_filt value.
        """
        # if jth column is 0 --> new cycle
        if pivots[j] == -1:
            pds = sum([len(S[cnt]) for cnt in range(dim)]) # previous dimension needed to be subtracted 
            a = (S[dim])[j-pds].int_filt
            pp_int.append((dim, (a,inf)))

        # if column is non-zero --> death of class (that must have been born previously)
        else:
            pds1 = sum([len(S[cnt]) for cnt in range(dim)]) # previous dimension needed to be subtracted 
            pds2 = sum([len(S[cnt]) for cnt in range(max(0,dim-1))])  # previous dimension needed to be subtracted 

            a = (S[dim-1])[pivots[j]-pds2].int_filt
            b = (S[dim])[j-pds1].int_filt
            pp_int.remove((dim-1, (a, inf)))
            pp_int.append((dim-1, (a,b)))


    # sort by beginning of interval
    firstof = lambda l: (l[1])[0]
    pp_int.sort(key=firstof)

    return pp_int


In [ ]:
def representatives(V, pp_int, pp_cont, S):
    """
    Takes a matrix V as given by column_red(), 
    persistence pairs pp_int with integer filtration values
    persistence pairs pp_cont with continuous filtration values
    and a list S of sublists S[i] each containing Simplex type objects. 
    
    Returns a list rep_list of Persistence_Pair objects.
    """
    
    # The j-th column of V encodes the columns in ∂ that add up to give the j-th column in R.  (p. 182 / 194)
    m,n = V.shape
    rep_list = []
    
    
    for index in range(len(pp_int)):
        (dim,(a,b)) = pp_int[index]
        (dim,(a_cont,b_cont)) = pp_cont[index]
        
        
        for i in range(len(S[dim])):
            if S[dim][i].int_filt == a:
                j = i
                break
                
        pds = sum([len(S[cnt]) for cnt in range(dim)])
        j += pds
        
        # We want to take the birth representative,
        # meaning we look at the column of the cycle being born
        
        jcolumn = V[:,j]
        reps = []
        
        for i in range(0,m): 
            if jcolumn[i] == 1: 
                S_index = i - pds
                Simplex = S[dim][S_index] 
                reps.append(Simplex)
        
        rep_list.append(sc.Persistence_Pair(dim, (a,b), (a_cont,b_cont), reps))
    
    

    return rep_list
    

In [ ]:
def disc2cont_pp(pp, filt):
    """
    Takes the list of persistence pairs with discrete filtration values, as well as the original filtration,
    and outputs a list of peristence pairs with their smooth filtration values. 
    """
    pp_cont = []
    for (dim, (a,b)) in pp:
        a_cont = (filt[a])[1]
        
        if b != float("inf"):
            b_cont = (filt[b])[1]
        else:
            b_cont = float("inf")
            
        pp_cont.append((dim, (a_cont, b_cont)))
        
    return pp_cont

## The final PH()

In [ ]:
def PH(filt, S):
    """
    Takes list "filt" of naive filtration with continuous filtration values and
    list "S" containing sublists of Simplex type objects, sorted by dimension
    
    Returns the dimensions of Cn, Bn, Zn and Hn, 
    as well as persistence pairs "pp" and representatives "reps" of homology classes.
    """
    
    
    d=2 # this is the dimension we are working in
        # right now we are only on the square, so d=2
    
    D = boundary_matrix(S)
    
    dim_list = [len(S[i]) for i in range(0,d+1)]
    
    
    pivots, R, V = column_red(D)
    
    
    
    inf = float('inf')
    

    dCn, dZn, dBn, dHn = calc_groups(dim_list, pivots)
    
    

    pp_int = calc_pps(D, dim_list, S)
    
    
    # persistance pairs with continuous values
    pp_cont = disc2cont_pp(pp_int, filt)

    # Caculate representatives (contains also information of pp_int and pp_cont)
    pairs_w_reps = representatives(V, pp_int, pp_cont, S) 
    
  
    
    
    for timestep in range(len(filt)):
        for v in S[0]:
            v.calc_cc(S, timestep)
    
    
    return dCn, dZn, dBn, dHn, pp_int, pp_cont, pairs_w_reps

# Calculating the evolution $\Lambda_0$

## Auxiliary functions for Lambda_0_evolution()

In [ ]:
def p_vol_3d(matrix_np):
    
    """
    Takes 3x3 numpy array containing 3 column vectors, 
    of which either p = 0, 1, 2 or 3 are linearly independant.
    Returns the p-dimensional volume the column vectors span, as well as the dimension p.
    """
    matrix = sp.Matrix(matrix_np)
    mat_red, pivots = matrix.T.rref()
    mat_red=mat_red.T
    p = len(pivots)
    
    if   p==0:
        det_p = 1
    
    elif p==1:
        #print("pivots:")
        #print(pivots)
        #print("reduced matrix:")
        #sp.pprint(mat_red)
        det_p = mat_red[:,0].norm()
    
    elif p==2:
        """
        !!!
        Take norm of cross product of the two reduced vectors
        !!!
        """
        
    elif p==3:
        det_p = la.det(matrix_np)
        
        
    return abs(det_p), p

## The final Lambda_0_evolution()

In [ ]:
def Lambda_0_evolution(p_filt, N, pairs_w_reps):
    """
    Takes naive filtration p_filt, 
    number of original connected components (number of 0-Simplices) N, 
    and list of persistence pairs pairs_w_reps.
    
    Returns numpy array time_list of size (len(p_filt), N, 2, 2),
    where the first index is the integer filtration step,
    the second index is the connected component,
    and the leftover 2x2 matrix is the basis spanning Lambda_0 of this connected component at a given timestep. 
    """

    # Make list of lists
    # First index gives timestep
    # Second index gives connected component (0 to N-1)
    # This then contains all the vectors contributing to this connected component. 
    from Reduce2IntegerBasis import reduce_spanning_set_2d as rss2
    #rss2 = reload(rss2)

    #time_list = [[[] for j in range(N)] for i in range(len(p_filt))]
    time_list = [[np.zeros((2,2),dtype=np.int32) for j in range(N)] for i in range(len(p_filt))]
    time_list = np.zeros((len(p_filt),N,2,2),dtype=np.int32)
    # We need to calculate the connected components .cc for each pair in pairs_w_reps at each timestep i. 
    # At this point we do not save a list of these connected components for each timestep
    # rather we look at some simplex that forms the cacly of the representative pair and look at its cc. 
    # this way we only have little computational work whenever we need to look this up
    # however, in a case such as here where we want to have this information at any given timestep
    # it might be easier to just calculate the cc's once and save them (?)


    for t in range(len(p_filt)):
        # we are at timestep "t"
        span_set_list = [[] for j in range(N)] # one list for each cc
        for pair in pairs_w_reps:
            # we look at persistence pair "pair"
            if pair.dim == 1 and pair.start_int <= t and pair.end_int > t:
                # if this persistence pair is of dimension 1 (1-homology), 
                # has been born before t and will die after t
                # (i.e. it is alive at time t)
                # then its crossing vector contributes to the crossing vectors 
                # of its connected component at time t
                pair.calc_cc() # calculate the connected component of pair
                cc = pair.cc[t]
                #(time_list[time])[cc].append(pair.return_cv(time))
                new_cv = pair.return_cv(t)
                if abs(la.norm(new_cv))>0.9:
                    # check if new vector is noan-trivial before calling the basis reduction function
                    span_set_list[cc] = rss2(span_set_list[cc],new_cv)

        for comp in range(N):
            for i in range(len(span_set_list[comp])):
                time_list[t][comp][:,i] = span_set_list[comp][i]


    """
    for t in range(N,len(p_filt)):
        print(f"\ntime {t}:")
        for comp in range(N):
            print(f"Component {comp} -- Lambda_0 spanned by: {(time_list[t])[comp]}")

    """
    
    return time_list

